In [4]:
#Bloque0 Instalación de la libreria pydeck
#https://deckgl.readthedocs.io/en/latest/installation.html
#pip install pydeck


In [1]:
#RECOMENDACIONES:
#Una vez bajado el proyecto, dirijirse a la documentación de pydeck e instalar todas las dependecias necesarias.
#Recomendamos abrir el proyecto en jupiter o google colab para evitar dificultades.
#Link de colab: https://colab.research.google.com/drive/1Zta-DxXfS5pVaC7_lG19Ib7rfMoBaOmo

# Bloque1 (Importación de librerias y de datos)
import pandas as pd
import pydeck as pdk
import numpy as np
import networkx as nx
archivo=pd.read_csv("https://raw.githubusercontent.com/mauriciotoro/ST0245-Eafit/master/proyecto/Datasets/calles_de_medellin_con_acoso.csv", sep=";")
poligono=pd.read_csv("https://raw.githubusercontent.com/mauriciotoro/ST0245-Eafit/master/proyecto/Datasets/poligono_de_medellin.csv", sep=";")



In [ ]:
#Bloque2 (Medir zonas con más alto riesgo)
acoso=archivo['harassmentRisk']>0.97
archivo2=archivo[acoso]

#Eliminamos columnas sobrantes
del(archivo2['name'])
del(archivo2['length'])
del(archivo2['oneway'])
del(archivo2['harassmentRisk'])
del(archivo2['geometry'])

#divimos las longitudes y latitudes de origen y destino 
archivo2['longitud_d']=archivo2['destination'].map(lambda x:x.split(',')[0])
archivo2['latitud_d']=archivo2['destination'].map(lambda x:x.split(',')[1])
archivo2['longitud_o']=archivo2['origin'].map(lambda x:x.split(',')[0])
archivo2['latitud_o']=archivo2['origin'].map(lambda x:x.split(',')[1])

#borramos las columnas de origin y destination
del(archivo2['origin'])
del(archivo2['destination'])
#eliminamos los parentesis y los pasamos a float
archivo2=archivo2.replace('\(|\)','',regex=True).astype(float)

#Nueva lista que indica el color de las calles de alto riesgo de acoso
color = []
for _ in range(7576):
   color.append("#ed1c24")

#Nueva lista con el nommbre que aparecera con pongamos el mouse sobre la ruta
name = []
for _ in range(7576):
   name.append("Alto riesgo de acoso")

#Lista final con el mayor número de acoso en las calles
acoso_o = list(zip(archivo2.longitud_o,archivo2.latitud_o))
acoso_d = list(zip(archivo2.longitud_d,archivo2.latitud_d))
acoso_M=list(zip(acoso_o,acoso_d))
mayor_acoso=list(zip(name,color,acoso_M))
mayor_acoso

In [3]:
#Bloque3 (organización de información y coordenadas del usuario)
distancia = nx.from_pandas_edgelist(archivo, source='origin', target='destination', edge_attr='length')

origin_long=float(input("Ingrese longitud del origen: "))
origin_lat=float(input("Ingrese latitud del origen: "))

#Coordenadas de origen
origen=(origin_long, origin_lat)

destination_long=float(input("Ingrese longitud del destino: "))
destination_lat=float(input("Ingrese latitud del destino: "))

#Coordenadas de destino
destino=(destination_long,destination_lat)



In [ ]:
#Bloque4 (Imprimir vertices o nodos de la grafica)
distancia.nodes()

In [ ]:
#Bloque5 (Imprimir las aristas de las distancias)
distancia.edges()

In [ ]:
#Bloque6 (Cantidad de vertices o nodos)
distancia.order()

In [ ]:
#Bloque7 (Implementacion de algoritmo dijkstra para ruta más corta en una lista de strings)
djk_distancia=nx.dijkstra_path(distancia, source=str(origen), target=str(destino), weight=True)
djk_distancia


In [ ]:
#Bloque8
#Pasamos a dataframe el resultado
dist = pd.DataFrame(djk_distancia,columns=['calles'])

# dividimos en dos columnas la longitud y la latitud
dist['longitud']=dist['calles'].map(lambda x:x.split(',')[0])
dist['latitud']=dist['calles'].map(lambda x:x.split(',')[1])

#borramos la columna de calles
del(dist['calles'])

#eliminamos los parentesis y los pasamos a float
dist=dist.replace('\(|\)','',regex=True).astype(float)

#hacemos una nueva lista con el sistemas de coordenadas tipo turple
points = list(zip(dist.longitud,dist.latitud))


points

In [ ]:
#Bloque9 (Nueva lista con información de ruta mas corta)
calles=[points]
name=["Distancia más corta"]
color=["#ff0080"]
camino=list(zip(name,color,calles))

camino

In [ ]:
#Bloque 10 (Nuevo dataframe de las rutas de mayor acoso)
ruta_acoso=pd.DataFrame(mayor_acoso, columns=['name','color','path'])
ruta_acoso

In [ ]:
#Bloque10 Nuevo dataframe para colorear la ruta más corta
distancia_corta = pd.DataFrame(camino, columns=['name','color','path'])
distancia_corta


In [ ]:
#Bloque11 Unión de los dataframe, para exportar datos en el mapa
rutas = [distancia_corta,ruta_acoso]
final=pd.concat(rutas)
final

In [ ]:
#Bloque12 (Exportar mapa)

#Función que lee el color de la ruta y la pinta
def ruta_color(d):
    d = d.lstrip("#")
    return tuple(int(d[i : i + 2], 16) for i in (0, 2, 4))


final["color"] = final["color"].apply(ruta_color)

#Coordenadas en donde aparece el mapa en el lugar de origen
coordenadas_map = pdk.ViewState(latitude=origin_lat, longitude=origin_long, zoom=18, bearing=0, pitch=45)

#Caracteristicas del mapa
layer = pdk.Layer(
    type="PathLayer",
    data=final,
    pickable=True,
    get_color="color",
    width_scale=2,
    width_min_pixels=2,
    get_path="path",
    get_width=2,
)  

#extraemos propiedades del mapa
mapa = pdk.Deck(layers=[layer], initial_view_state = coordenadas_map, tooltip={"text": "{name}"})

#pasamos el mapa a html
mapa.to_html("map.html")